In [8]:
!pip install optuna --quiet

# 🚗 Paso 0: Montar Google Drive e importar librerías
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm

# ⚙️ Paso 1: Configuración
ruta_entrada = "/content/drive/MyDrive/Datos/8_Base_Funcional_Reducida_RQ.parquet"
ruta_salida = "/content/drive/MyDrive/Datos/9_1EspacioF.parquet"
n_ventana = 5
anio_max = 2023
tamaño_muestra_optuna_k = 0.10
tamaño_muestra_optuna_pesos = 0.10
n_trials_optuna_k = 30
n_trials_optuna_pesos = 30
anios_validos = list(range(anio_max - 24, anio_max + 1))  # 25 años hacia atrás: 1999–2023

# 📥 Paso 2: Cargar la base funcional original
base = pd.read_parquet(ruta_entrada).drop_duplicates()
print(f"✅ Base cargada: {base.shape}")

# 🔍 Paso 3: Detectar último año con datos por empresa
columnas_anio = [col for col in base.columns if re.match(r".+_\d{4}$", col)]
df_anios = base[columnas_anio].copy()

ultimos_anios = []
for idx, fila in tqdm(df_anios.iterrows(), total=df_anios.shape[0], desc="🧠 Detectando año final por empresa"):
    anios = [int(col.split("_")[-1]) for col in fila.index if pd.notna(fila[col]) and int(col.split("_")[-1]) in anios_validos]
    ultimos_anios.append(max(anios) if anios else None)

serie_ultimos_anios = pd.Series(ultimos_anios, index=base.index)
empresas_validas = base[serie_ultimos_anios.notna()].copy()
serie_ultimos_anios = serie_ultimos_anios.dropna().astype(int)

print(f"✅ Empresas con último año válido: {empresas_validas.shape[0]:,}")

# 🧠 Paso 4: Crear RQ_final con base en último año disponible
rq_final = []
for idx, fila in empresas_validas.iterrows():
    anio_rq = serie_ultimos_anios.loc[idx]
    col_rq = f"RQ_{anio_rq}"
    rq_valor = fila[col_rq] if col_rq in fila and pd.notna(fila[col_rq]) else np.nan
    rq_final.append(rq_valor)

empresas_validas["RQ_final"] = rq_final
empresas_validas = empresas_validas[empresas_validas["RQ_final"].notna()].copy()
empresas_validas["RQ_final"] = empresas_validas["RQ_final"].astype(int)

print(f"✅ Empresas con RQ_final asignado: {empresas_validas.shape[0]:,}")

# 🔄 Paso 5: Construcción del espacio funcional con ventanas móviles
columnas_anio = [col for col in empresas_validas.columns if re.match(r".+_\d{4}$", col)]
indicadores = sorted(set(col.split("_")[0] for col in columnas_anio if not col.startswith("RQ")))
columnas_extra = ["DEP", "CIIU_Letra"]  # columnas adicionales que queremos conservar

data_ventanas = []
for idx, fila in tqdm(empresas_validas.iterrows(), total=empresas_validas.shape[0], desc="🔧 Construyendo trayectorias funcionales"):
    anio_final = serie_ultimos_anios.loc[idx]
    anios_ventana = list(range(anio_final - n_ventana + 1, anio_final + 1))
    fila_nueva = {
        "NIT": idx,
        "RQ_final": fila["RQ_final"],
        "Año_final": anio_final
    }

    # Agregar columnas extra (si están disponibles)
    for col in columnas_extra:
        if col in fila:
            fila_nueva[col] = fila[col]

    # Agregar trayectorias funcionales
    for var in indicadores:
        for i, anio in enumerate(anios_ventana[::-1]):  # de más antiguo (-4) a más reciente (-0)
            col_original = f"{var}_{anio}"
            col_nueva = f"{var}_-{n_ventana - 1 - i}"
            fila_nueva[col_nueva] = fila.get(col_original, np.nan)

    data_ventanas.append(fila_nueva)


# 📊 Paso 6: Crear DataFrame final del espacio funcional
espacioF = pd.DataFrame(data_ventanas).set_index("NIT")
print(f"✅ Espacio funcional generado con forma: {espacioF.shape}")

# 💾 Paso 7: Guardar base funcional con ventanas móviles
espacioF.to_parquet(ruta_salida)
print(f"💾 Base guardada en: {ruta_salida}")


# 🧽 Limpiar y filtrar
df = espacioF.copy()
df = df[df["RQ_final"].notna()].copy()


# 📊 Seleccionar columnas funcionales
columnas_funcionales = [col for col in df.columns if re.match(r".+_-\d$", col)]
indicadores = sorted(set(col.split("_")[0] for col in columnas_funcionales))

# --------------------------------------------------
# 🧠 Paso 1: Optimización de k y lambda con Optuna
# --------------------------------------------------
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm

muestra_k = df.sample(frac=tamaño_muestra_optuna_k, random_state=123)
X_k = muestra_k[columnas_funcionales].copy()
y_k = muestra_k["RQ_final"].copy()

def distancia_funcional(f1, f2, lambda_p, n=n_ventana):
    total = 0
    for var in indicadores:
        v1 = [f1.get(f"{var}_-{i}", np.nan) for i in range(n)]
        v2 = [f2.get(f"{var}_-{i}", np.nan) for i in range(n)]

        l1 = 0
        validos = 0
        for a, b in zip(v1, v2):
            if pd.notna(a) and pd.notna(b):
                if np.isinf(a) and np.isinf(b) and a == b:
                    l1 += 0  # inf vs inf del mismo signo
                elif np.isinf(a) or np.isinf(b):
                    l1 += np.inf
                else:
                    l1 += abs(a - b)
                validos += 1

        faltantes = n - validos
        if validos > 0:
            penalizada = l1 * (1 + lambda_p * (faltantes / n))
            acotada = penalizada / (1 + penalizada) if np.isfinite(penalizada) else 1.0
        else:
            acotada = 1.0

        total += acotada

    return total / len(indicadores)


def objective(trial):
    k = trial.suggest_int("k", 3, 15)
    lambda_p = trial.suggest_float("lambda", 0.1, 5.0)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    f1s = []

    for train_idx, test_idx in tqdm(skf.split(X_k, y_k), total=5, desc="↪ Validación cruzada (k, λ)", leave=False):
        X_train, X_test = X_k.iloc[train_idx], X_k.iloc[test_idx]
        y_train, y_test = y_k.iloc[train_idx], y_k.iloc[test_idx]

        preds = []
        for i, fila_test in X_test.iterrows():
            dists = [(distancia_funcional(fila_test, X_train.iloc[j], lambda_p), y_train.iloc[j])
                     for j in range(len(X_train))]
            vecinos = sorted(dists, key=lambda x: x[0])[:k]
            pred = round(np.mean([v[1] for v in vecinos]))
            preds.append(pred)

        f1s.append(f1_score(y_test, preds))

    return np.mean(f1s)


print("🔧 Optimizando k y lambda con Optuna...")

import logging
optuna.logging.set_verbosity(optuna.logging.WARNING)

study_k = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
for _ in tqdm(range(n_trials_optuna_k), desc="🔍 Buscando k y λ"):
    study_k.optimize(objective, n_trials=1, catch=(Exception,))

k_optimo = study_k.best_params["k"]
lambda_optimo = study_k.best_params["lambda"]
print(f"\n✅ k óptimo: {k_optimo}, λ óptimo: {lambda_optimo:.4f}, F1-score: {study_k.best_value:.4f}")


# --------------------------------------------------
# ⚖️ Paso 2: Optimización de pesos con Optuna
# --------------------------------------------------
muestra_p = df.sample(frac=tamaño_muestra_optuna_pesos, random_state=222)
X_p = muestra_p[columnas_funcionales].copy()
y_p = muestra_p["RQ_final"].copy()

def distancia_ponderada(f1, f2, lambda_p, n, pesos):
    total, suma_pesos = 0, 0
    for var in indicadores:
        v1 = [f1.get(f"{var}_-{i}", np.nan) for i in range(n)]
        v2 = [f2.get(f"{var}_-{i}", np.nan) for i in range(n)]

        l1, validos = 0, 0
        for a, b in zip(v1, v2):
            if pd.notna(a) and pd.notna(b):
                if np.isinf(a) and np.isinf(b) and a == b:
                    l1 += 0
                elif np.isinf(a) or np.isinf(b):
                    l1 += np.inf
                else:
                    l1 += abs(a - b)
                validos += 1

        faltantes = n - validos
        if validos > 0:
            penalizada = l1 * (1 + lambda_p * (faltantes / n))
            acotada = penalizada / (1 + penalizada) if np.isfinite(penalizada) else 1.0
        else:
            acotada = 1.0

        total += pesos[var] * acotada
        suma_pesos += pesos[var]

    return total / suma_pesos if suma_pesos > 0 else 1.0

def objective_pesos(trial):
    pesos = {var: trial.suggest_float(f"peso_{var}", 0.1, 5.0) for var in indicadores}
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    f1s = []

    for train_idx, test_idx in tqdm(skf.split(X_p, y_p), total=5, desc="↪ Validación cruzada (pesos)", leave=False):
        X_train, X_test = X_p.iloc[train_idx], X_p.iloc[test_idx]
        y_train, y_test = y_p.iloc[train_idx], y_p.iloc[test_idx]

        preds = []
        for i, fila_test in X_test.iterrows():
            dists = [(distancia_ponderada(fila_test, X_train.iloc[j], lambda_optimo, n_ventana, pesos), y_train.iloc[j])
                     for j in range(len(X_train))]
            vecinos = sorted(dists, key=lambda x: x[0])[:k_optimo]
            pred = round(np.mean([v[1] for v in vecinos]))
            preds.append(pred)

        f1s.append(f1_score(y_test, preds))

    return np.mean(f1s)



print("\n⚖️ Optimizando pesos con Optuna...")

import logging
optuna.logging.set_verbosity(optuna.logging.WARNING)

study_pesos = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
for _ in tqdm(range(n_trials_optuna_pesos), desc="🔍 Buscando pesos"):
    study_pesos.optimize(objective_pesos, n_trials=1, catch=(Exception,))


# 🏁 Resultados finales
mejores_pesos = {k.replace("peso_", ""): v for k, v in study_pesos.best_params.items()}
top_3 = sorted(mejores_pesos.items(), key=lambda x: -x[1])[:3]

print("\n🏆 Top 3 indicadores más importantes:")
for var, peso in top_3:
    print(f"🔹 {var}: {peso:.4f}")
print(f"\n✅ F1-score final con pesos óptimos: {study_pesos.best_value:.4f}")

# 💾 Guardar resultados en 9_2ParametrosFuncional.pkl
import pickle

parametros_optimos = {
    "k": k_optimo,
    "lambda": lambda_optimo,
    "pesos": mejores_pesos,
    "top3": top_3
}

with open("/content/drive/MyDrive/Datos/9_2ParametrosFuncional.pkl", "wb") as f:
    pickle.dump(parametros_optimos, f)

print("📁 Parámetros óptimos guardados en: 9_2ParametrosFuncional.pkl")

# 📝 Crear resumen de salida
with open("/content/drive/MyDrive/Datos/9_2Resumen.txt", "w") as f:
    f.write("📄 Resumen de procesamiento y optimización del modelo funcional\n")
    f.write("=================================================================\n\n")

    f.write("✅ Se construyó el espacio funcional ℱ con trayectorias financieras móviles\n")
    f.write(f"   - Archivo generado: 9_1EspacioF.parquet\n")
    f.write(f"   - Número de empresas: {espacioF.shape[0]:,}\n")
    f.write(f"   - Número de columnas funcionales: {len(columnas_funcionales)}\n")
    f.write(f"   - Columnas adicionales incluidas: DEP, CIIU_Letra, Año_final\n\n")

    f.write("🧠 Optimización de hiperparámetros:\n")
    f.write(f"   - k óptimo: {k_optimo}\n")
    f.write(f"   - λ óptimo: {lambda_optimo:.4f}\n")

    f.write("\n⚖️ Optimización de pesos por indicador:\n")
    for var, peso in top_3:
        f.write(f"   - {var}: {peso:.4f}\n")

    f.write(f"\n🎯 F1-score final con pesos óptimos: {study_pesos.best_value:.4f}\n\n")
    f.write("💾 Parámetros guardados en: 9_2ParametrosFuncional.pkl\n")

from IPython.display import display
display(pd.DataFrame([parametros_optimos['top3']], index=["Top 3 Indicadores"]))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Base cargada: (5768, 524)


🧠 Detectando año final por empresa:   0%|          | 0/5768 [00:00<?, ?it/s]

✅ Empresas con último año válido: 5,565
✅ Empresas con RQ_final asignado: 5,565


🔧 Construyendo trayectorias funcionales:   0%|          | 0/5565 [00:00<?, ?it/s]

✅ Espacio funcional generado con forma: (5565, 89)
💾 Base guardada en: /content/drive/MyDrive/Datos/9_1EspacioF.parquet
🔧 Optimizando k y lambda con Optuna...


...

↪ Validación cruzada (pesos):   0%|          | 0/5 [00:00<?, ?it/s]

↪ Validación cruzada (pesos):   0%|          | 0/5 [00:00<?, ?it/s]

↪ Validación cruzada (pesos):   0%|          | 0/5 [00:00<?, ?it/s]


🏆 Top 3 indicadores más importantes:
🔹 RAO: 4.9635
🔹 RCC: 4.8677
🔹 ROI: 4.7665

✅ F1-score final con pesos óptimos: 0.9147
📁 Parámetros óptimos guardados en: 9_2ParametrosFuncional.pkl


,0,1,2
Top 3 Indicadores,"(RAO, 4.963522674168568)","(RCC, 4.867683255568788)","(ROI, 4.766492118974347)"
